In [1]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from tqdm import tqdm
from pdf_ingestor import extract_and_split_pdf
from pdf_ingestor import extract_and_split_pdf_v2
from ytb_ingestor import extract_and_split_ytb
from vectorstore import index_documents, index_documents_v2, create_vectorstore
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
import os 
from logger import get_logger
logger = get_logger()

c:\Users\habou\miniconda3\envs\genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embedding_model = HuggingFaceBgeEmbeddings(
    model_name="../../bge-m3",
    encode_kwargs={"normalize_embeddings": True},
    query_instruction="query: "
)

C:\Users\habou\AppData\Local\Temp\ipykernel_25204\961953646.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(


In [3]:
embedding_model_2 = HuggingFaceBgeEmbeddings(
    model_name="../../multilingual-e5-large",
    encode_kwargs={"normalize_embeddings": True},
    query_instruction="query: "
)

In [ ]:
"""index_path = "../../vectorstore/faiss_vectorestore_v2"
create_vectorstore(embedding_model, index_path)"""

[20:14:17.946] INFO - Création d'un index FAISS vide (dim = 1024)
[20:14:18.053] INFO - Sauvegarde de l'index vide dans: ../../vectorstore/faiss_vectorestore_v2
[20:14:18.055] INFO - Index vide (cosinus) créé avec succès.


In [ ]:

dict_ytb_videos_urls = {
    "PLAN STRATEGIQUE RENAULUTION" : "https://www.youtube.com/watch?app=desktop&v=EtivAvmDr2Q&t=901s",
    "Conférence résultats financiers 2021 de Renault Group" : "https://www.youtube.com/watch?v=VfIeaIFSCQA",
    "Conférence résultats financiers 2022 de Renault Group" : "https://www.youtube.com/watch?v=UWHlyjVtwT8",
    "Conférence résultats financiers 2023 de Renault Group" : "https://www.youtube.com/watch?v=B57wephix-w",
    "Conférence résultats financiers 2024 de Renault Group" : "https://www.youtube.com/watch?v=BA5ZOtWfpY0",
}

In [ ]:
"""pdf_chunks = []
ytb_chunks = []
CHUNK_SIZE = 450
OVERLAP = 64

for filename in (pdf_files_list) : 
    file_path = os.path.join(folder_path, filename)
    pdf_chunks.extend(extract_and_split_pdf(file_path, CHUNK_SIZE, OVERLAP))

for video_name, url in dict_ytb_videos_urls.items() : 
    ytb_chunks.extend(extract_and_split_ytb(url, video_name, CHUNK_SIZE, OVERLAP))

all_chunks = pdf_chunks + ytb_chunks"""

'pdf_chunks = []\nytb_chunks = []\nCHUNK_SIZE = 450\nOVERLAP = 64\n\nfor filename in (pdf_files_list) : \n    file_path = os.path.join(folder_path, filename)\n    pdf_chunks.extend(extract_and_split_pdf(file_path, CHUNK_SIZE, OVERLAP))\n\nfor video_name, url in dict_ytb_videos_urls.items() : \n    ytb_chunks.extend(extract_and_split_ytb(url, video_name, CHUNK_SIZE, OVERLAP))\n\nall_chunks = pdf_chunks + ytb_chunks'

In [19]:
CHUNK_SIZE = 150
OVERLAP = 25
folder_path = "../../data"
markdown_folder = "../../data/md_extracted"
pdf_files_list = ["Renault DEU 2020.pdf", "Renault DEU 2021.pdf", "Renault URD 2022.pdf", "Renault URD 2023.pdf", "Renault 2024 Rapport d’activité.pdf"]
pdf_chunks = []

In [ ]:
for filename in (pdf_files_list) : 
    file_path = os.path.join(folder_path, filename)
    md_file_path = os.path.join(markdown_folder, filename.split('.')[0]+'.md')
    chunks = extract_and_split_pdf_v2(file_path, md_file_path, CHUNK_SIZE, OVERLAP)
    pdf_chunks.extend(chunks)

In [ ]:
len(pdf_chunks)
cop_dpdf = pdf_chunks

12488

In [ ]:
pdf_2020 = pdf_chunks[0:3038]
pdf_2021 = pdf_chunks[3038:6234]
pdf_2022 = pdf_chunks[6234:9416]
pdf_2023 = pdf_chunks[9416:12341]
pdf_2024 = pdf_chunks[12341:]

In [40]:

import uuid
from langchain_core.documents import Document

def chunk_correction(all_pdf_chunks, filename):
    all_chunks = []
    for chunk in all_pdf_chunks : 
        all_chunks.append(Document(
                page_content = chunk,
                metadata={
                    "source": filename,
                    "id": str(uuid.uuid4())
                }
            ))
    return all_chunks

In [50]:
chunk_pdf_2020 = chunk_correction(pdf_2020, "Renault 2020")
chunk_pdf_2021 = chunk_correction(pdf_2021, "Renault 2021")
chunk_pdf_2022 = chunk_correction(pdf_2022, "Renault 2022")
chunk_pdf_2023 = chunk_correction(pdf_2023, "Renault 2023")
chunk_pdf_2024 = chunk_correction(pdf_2024, "Renault 2024")

In [59]:
len(chunk_pdf_2024)

147

In [60]:
final_chunks = chunk_pdf_2020 + chunk_pdf_2021 + chunk_pdf_2022 + chunk_pdf_2023 + chunk_pdf_2024

In [5]:
import pickle


with open("../../vectorstore/documents_v2.pkl", "rb") as f:
    final_chunks = pickle.load(f)

In [23]:
index_path = "../../vectorstore/faiss_vectorestore_v2"
create_vectorstore(embedding_model_2, index_path)

[20:40:14.804] INFO - Création d'un index FAISS vide (dim = 1024)
[20:40:14.916] INFO - Sauvegarde de l'index vide dans: ../../vectorstore/faiss_vectorestore_v2
[20:40:14.918] INFO - Index vide (cosinus) créé avec succès.


In [27]:
from vectorstore import index_documents
from vectorstore_v2 import index_documents_optimized_bge_m3, index_documents_ultra_optimized_bge_m3, index_documents_with_multiprocessing, index_documents_xps13_complete
index_documents_xps13_complete(final_chunks[0:100], embedding_model_2, "../../vectorstore/faiss_vectorestore_v2", run_diagnostics=False)

ImportError: cannot import name 'index_documents_xps13_complete' from 'vectorstore_v2' (c:\Users\habou\OneDrive\Bureau\python-projects\renaulution-rag\backend\data_ingestion\vectorstore_v2.py)

In [65]:
from langchain.vectorstores import FAISS

# Charge l'index local
vectorstore = FAISS.load_local("../../vectorstore/faiss_vectorestore_v2", embedding_model, allow_dangerous_deserialization=True)

# Récupère le nombre de documents indexés
nb_docs = len(vectorstore.index_to_docstore_id)
print(f"📦 Nombre de documents indexés : {nb_docs}")


📦 Nombre de documents indexés : 0


In [ ]:
chunk_pdf_2020

In [3]:
"""ytb_chunks = []
CHUNK_SIZE = 450
OVERLAP = 64

for video_name, url in dict_ytb_videos_urls.items() : 
    ytb_chunks.extend(extract_and_split_ytb(url, video_name, CHUNK_SIZE, OVERLAP))"""

'ytb_chunks = []\nCHUNK_SIZE = 450\nOVERLAP = 64\n\nfor video_name, url in dict_ytb_videos_urls.items() : \n    ytb_chunks.extend(extract_and_split_ytb(url, video_name, CHUNK_SIZE, OVERLAP))'

In [22]:
index_documents(pdf_chunks, embedding_model, "../../vectorstore/faiss_vectorestore_v2")

[02:17:49.285] INFO - Début de l’indexation de 12488 chunks en batches de 64...
[02:17:49.287] INFO - Calcul des embeddings et indexation par batch avec barre de progression...
Indexing:   0%|          | 0/196 [00:00<?, ?batch/s]
[02:17:49.293] ERROR - Erreur lors de l’indexation : 'str' object has no attribute 'id'
Traceback (most recent call last):
  File "c:\Users\habou\OneDrive\Bureau\python-projects\renaulution-rag\backend\data_ingestion\vectorstore.py", line 44, in index_documents
    vectorstore.add_documents(batch)
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^
  File "c:\Users\habou\miniconda3\envs\genai\Lib\site-packages\langchain_core\vectorstores\base.py", line 279, in add_documents
    ids = [doc.id for doc in documents]
           ^^^^^^
AttributeError: 'str' object has no attribute 'id'


In [15]:
"""pdf_chunks = extract_and_split_pdf(file_path, 450, 64)
ytb_chunks = extract_and_split_ytb("https://www.youtube.com/watch?v=VfIeaIFSCQA", 450, 64)"""

'pdf_chunks = extract_and_split_pdf(file_path, 450, 64)\nytb_chunks = extract_and_split_ytb("https://www.youtube.com/watch?v=VfIeaIFSCQA", 450, 64)'

In [67]:
len(final_chunks)

12488

In [ ]:
"""import pickle

with open("../../vectorstore/documents_v2.pkl", "wb") as f:
    pickle.dump(final_chunks, f)"""

In [ ]:
"""vectorstore = FAISS.load_local(index_path, embedding_model, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("(Board of Management - BoM)", k=10)"""

In [ ]:
"""lexical_retriever = BM25Retriever.from_documents(pdf_chunks)
lexical_retriever.k = 3
lexical_retriever.get_relevant_documents("Identify the members of the board of directors in 2021.")"""

In [30]:
file_path = "../../data/pdf_test.pdf"
md_path = "./extrait_renault.md"

In [ ]:
chunks = extract_and_split_pdf_v2(file_path, md_path, 150, 20)

[17:14:12.344] INFO - Chargement du PDF : ../../data/pdf_test.pdf
c:\Users\habou\miniconda3\envs\genai\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\habou\miniconda3\envs\genai\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
[17:15:02.160] INFO - Markdown sauvegardé dans : ./extrait_renault.md
[17:15:02.163] INFO - Découpage initial du markdown en sections...
[17:15:02.164] INFO - 29 sections initialement détectées.
[17:15:02.165] INFO - 19 sections finales après fusion.
[17:15:02.166] INFO - Début du découpage par section avec gestion des tables...
[17:15:02.167] INFO - Texte découpé en 1 chunk(s).
[17:15:02.168] INFO - Texte découpé en 1 chunk(s).
[17:15:02.169] INFO - Texte

In [57]:
(chunks)

["## Présentation du Conseil d'administration\nAu 1 er mars 2021 ADMINISTRATEURS représentant les salariés Éric PERSONNE Richard GENTIL ADMINISTRATEUR représentant les salariés actionnaires ADMINISTRATEUR en qualité de représentant de l'État français Frédéric BARRAT Benoît OSTERTAG Martin VIAL ADMINISTRATEUR élu par l'Assemblée générale sur proposition de l'État français Jean-Dominique SENARD Président du Conseil ADMINISTRATEURS 58,5 ÂGE MOYEN NATIONALITÉS Thomas COURBE",
 "## Les comités spécialisés\nADMINISTRATEURS élus par l'Assemblée générale sur proposition de Nissan Yu SERIZAWA Pierre FLEURIOT* 4,3 66,7% (1) ANCIENNETÉ INDÉPENDANCE dont 3 présidentes de comités Patrick THOMAS Marie-Annick DARMAILLAC Joji TAGAWA ADMINISTRATEURS INDÉPENDANTS élus par l'Assemblée générale des actionnaires Pascale SOURISSE Catherine BARBA Miriem BENSALAH CHAQROUN Annette WINKLER * Administrateur référent Comité de L'ÉTHIQUE ET DE LA RSE (1) Hors administrateurs représentant les salariés et administra

In [58]:
chunk_text = """" "## Dacia-Lada, Tout. Simplement\nDacia, qui reste Dacia avec une touche de fraicheur et Lada, toujours robuste et solide, continueront à proposer des produits abordables, basés sur des technologies éprouvées et destinés aux acheteurs cherchant l'achat malin, tout en allant sur le segment C. - Des modèles économiques très efficaces : P - design-to-cost, P - efficacité : de 4 plateformes à 1 ; de 18 types de carrosserie à 11, P permettant d'atteindre en moyenne 1,1 million d'unités par plateforme contre 0,3 million d'unités ; - L'apparition d'une nouvelle gamme de produits compétitifs et P arrivée dans le segment C : - 7",
"""

In [59]:
from tiktoken import get_encoding

def count_tokens(text: str, model_name: str = "cl100k_base") -> int:
    encoding = get_encoding(model_name)
    return encoding.encode(text)

# Exemple :
tokens = count_tokens(chunk_text)
print(len(tokens))
print(tokens)



189
[1, 330, 567, 423, 28139, 8288, 2649, 11, 80905, 13, 4567, 2833, 198, 35, 28139, 11, 7930, 66167, 423, 28139, 9952, 6316, 15119, 1557, 409, 6816, 12333, 324, 1880, 445, 2649, 11, 44093, 22514, 68, 1880, 2092, 579, 11, 2018, 8977, 546, 3869, 10045, 261, 951, 54021, 671, 541, 4893, 11, 3122, 5512, 1765, 951, 14645, 50510, 583, 85, 14014, 1880, 52087, 5512, 10253, 34361, 7870, 1759, 23008, 11943, 326, 6, 97264, 8811, 258, 11, 16968, 665, 682, 519, 1765, 514, 10449, 356, 13, 482, 3959, 1491, 85798, 82653, 8467, 25945, 31914, 2492, 551, 393, 482, 2955, 4791, 41238, 11, 393, 482, 31914, 582, 13109, 551, 409, 220, 19, 12235, 630, 288, 3869, 220, 16, 2652, 409, 220, 972, 4595, 409, 1841, 2177, 27197, 3869, 220, 806, 11, 393, 29557, 61512, 294, 6, 63734, 71933, 665, 52138, 26193, 220, 16, 11, 16, 3610, 294, 6, 3928, 5512, 1370, 12235, 76701, 45973, 220, 15, 11, 18, 3610, 294, 6, 3928, 5512, 2652, 482, 445, 53149, 277, 684, 294, 31109, 48659, 9192, 2727, 409, 54021, 75332, 275, 22589, 1880, 

In [61]:
len(chunk_text.split())

105